In [371]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [372]:
property_df = pd.read_csv('data/cleaned_monthly_avg_prices.csv')
arrest_df = pd.read_csv('data/cleaned_arrest_data.csv')
noise_df = pd.read_csv('data/cleaned_noise_data.csv')
demographic_df = pd.read_csv('data/cleaned_demographic_data.csv')
poi_df = pd.read_csv('data/cleaned_POI.csv')

In [373]:
# Rename columns for consistency
demographic_df = demographic_df.rename(columns={'year': 'Year'})
property_df = property_df.rename(columns={'ZipCode': 'zipcode'})
arrest_df = arrest_df.rename(columns={'ZipCode': 'zipcode'})
noise_df = noise_df.rename(columns={'ZipCode': 'zipcode'})
demographic_df = demographic_df.rename(columns={'ZipCode': 'zipcode', 'year': 'Year'})
poi_df = poi_df.rename(columns={'ZIP_CODE': 'zipcode'})

In [374]:
# Preprocess property data
property_df['YearMonth'] = pd.to_datetime(property_df['YearMonth'])
property_df['Year'] = property_df['YearMonth'].dt.year
property_df['Month'] = property_df['YearMonth'].dt.month

In [375]:
# Preprocess arrest and noise data
for df in [arrest_df, noise_df]:
    df['YearMonth'] = pd.to_datetime(df['YearMonth'])
    df['Year'] = df['YearMonth'].dt.year
    df['Month'] = df['YearMonth'].dt.month

In [376]:
demographic_df['Year'] = pd.to_datetime(demographic_df['Year'].astype(str) + '-01-01').dt.tz_localize(None)

In [377]:
def extend_dataframe(df, end_date='2024-05-31', date_column='YearMonth'):
    end_date = pd.to_datetime(end_date).tz_localize(None)
    
    if date_column == 'Year':
        df[date_column] = pd.to_datetime(df[date_column].astype(str) + '-01-01')
        last_date = df[date_column].max().tz_localize(None)
        date_range = pd.date_range(start=last_date + pd.DateOffset(years=1), end=end_date, freq='YS')
    else:
        df[date_column] = pd.to_datetime(df[date_column]).dt.tz_localize(None)
        last_date = df[date_column].max()
        date_range = pd.date_range(start=last_date + pd.DateOffset(months=1), end=end_date, freq='MS')
    
    extension = pd.DataFrame({date_column: date_range})
    extension['Year'] = extension[date_column].dt.year
    if date_column != 'Year':
        extension['Month'] = extension[date_column].dt.month
    
    extended_df = df.copy()
    for zipcode in df['zipcode'].unique():
        last_row = df[df['zipcode'] == zipcode].iloc[-1]
        temp_df = extension.copy()
        temp_df['zipcode'] = zipcode
        for col in df.columns:
            if col not in [date_column, 'Year', 'Month', 'zipcode']:
                temp_df[col] = last_row[col]
        extended_df = pd.concat([extended_df, temp_df], ignore_index=True)
    
    if date_column == 'Year':
        extended_df['Month'] = 1  # Set Month to January for yearly data
    
    return extended_df

In [378]:
end_date = '2024-05-31'

# Extend datasets to the end date
arrest_df = extend_dataframe(arrest_df, end_date)
noise_df = extend_dataframe(noise_df, end_date)
demographic_df = extend_dataframe(demographic_df, end_date, date_column='Year')

# For POI data, we'll replicate it for each month from the start date to the end date
date_range = pd.date_range(start='2017-01-01', end=end_date, freq='MS')
poi_df_extended = pd.DataFrame()

C:\Users\simon\AppData\Local\Temp\ipykernel_34160\1546782281.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[date_column] = pd.to_datetime(df[date_column].astype(str) + '-01-01')


In [379]:
for zipcode in poi_df['zipcode'].unique():
    temp_df = poi_df[poi_df['zipcode'] == zipcode].iloc[0].to_dict()
    temp_df = pd.DataFrame([temp_df] * len(date_range))
    temp_df['YearMonth'] = date_range
    temp_df['Year'] = temp_df['YearMonth'].dt.year
    temp_df['Month'] = temp_df['YearMonth'].dt.month
    poi_df_extended = pd.concat([poi_df_extended, temp_df], ignore_index=True)

poi_df = poi_df_extended

In [380]:
# Aggregate POI data by ZIP_CODE
poi_agg = poi_df.groupby('zipcode').agg({
    'count': 'sum',
    'distance_to_facility': 'mean',
    'cultural_facility': 'sum',
    'education_facility': 'sum',
    'health_services': 'sum',
    'public_safety': 'sum',
    'recreational_facility': 'sum',
    'religious_institution': 'sum',
    'transportation_facility': 'sum'
}).reset_index()

In [381]:
# Merge property and arrest data
merged_df = property_df.merge(arrest_df, on=['zipcode', 'Year', 'Month'], how='left')

In [382]:
# Merge noise data
merged_df = merged_df.merge(noise_df, on=['zipcode', 'Year', 'Month'], how='left')

In [383]:
merged_df = merged_df.rename(columns={'zipcode': 'zipcode'})

In [384]:
# Merge POI data
merged_df = merged_df.merge(poi_agg, on='zipcode', how='left')

In [385]:
# Rename YearMonth column in extended_demographic_df to avoid conflicts
extended_demographic_df.rename(columns={'YearMonth': 'DemographicYearMonth'}, inplace=True)

# Merge extended demographic data
merged_df = merged_df.merge(extended_demographic_df, on=['zipcode', 'Year', 'Month'], how='left')

In [386]:
merged_df

,zipcode,Borough,YearMonth_x,AveragePrice,Year,Month,YearMonth_y,ArrestCount,YearMonth,NoiseComplaints,...,cultural_facility,education_facility,health_services,public_safety,recreational_facility,religious_institution,transportation_facility,DemographicYearMonth,population,average_household_income
0,10001,Manhattan,2002-05-31,1053087,2002,5,NaT,NaN,NaT,NaN,...,0.0,89.0,0.0,0.0,0.0,0.0,0.0,NaT,NaN,NaN
1,10001,Manhattan,2010-11-30,1591637,2010,11,2010-11-01,101.0,2010-11-01,144.0,...,0.0,89.0,0.0,0.0,0.0,0.0,0.0,2010-11-01,18158.000000,128706.559600
2,10001,Manhattan,2021-12-31,1809826,2021,12,2021-12-01,46.0,NaT,NaN,...,0.0,89.0,0.0,0.0,0.0,0.0,0.0,2021-12-01,22491.088889,203201.673018
3,10001,Manhattan,2010-10-31,1600287,2010,10,2010-10-01,125.0,2010-10-01,173.0,...,0.0,89.0,0.0,0.0,0.0,0.0,0.0,2010-10-01,18158.000000,128706.559600
4,10001,Manhattan,2001-06-30,923270,2001,6,NaT,NaN,NaT,NaN,...,0.0,89.0,0.0,0.0,0.0,0.0,0.0,NaT,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50391,11694,Queens,2014-10-31,525009,2014,10,2014-10-01,32.0,2014-10-01,38.0,...,0.0,0.0,0.0,0.0,0.0,0.0,89.0,2014-10-01,15270.000000,90293.571400
50392,11694,Queens,2018-02-28,678188,2018,2,2018-02-01,20.0,2018-02-01,51.0,...,0.0,0.0,0.0,0.0,0.0,0.0,89.0,2018-02-01,15680.000000,94607.675200
50393,11694,Queens,2001-01-31,392616,2001,1,NaT,NaN,NaT,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,89.0,NaT,NaN,NaN
50394,11694,Queens,2005-01-31,547607,2005,1,NaT,NaN,NaT,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,89.0,NaT,NaN,NaN


In [387]:
# Drop duplicate columns and redundant 'YearMonth_x' column
merged_df = merged_df.drop(columns=['YearMonth_y', 'count'])
merged_df.rename(columns={'YearMonth_x': 'YearMonth'}, inplace=True)

In [388]:
merged_df.isnull().sum()

zipcode                         0
Borough                         0
YearMonth                       0
AveragePrice                    0
Year                            0
Month                           0
ArrestCount                 21927
YearMonth                   26873
NoiseComplaints             26873
distance_to_facility          293
cultural_facility             293
education_facility            293
health_services               293
public_safety                 293
recreational_facility         293
religious_institution         293
transportation_facility       293
DemographicYearMonth        20813
population                  20813
average_household_income    20813
dtype: int64

In [389]:
merged_df.dropna(inplace=True)

In [390]:
merged_df.isnull().sum()

zipcode                     0
Borough                     0
YearMonth                   0
AveragePrice                0
Year                        0
Month                       0
ArrestCount                 0
YearMonth                   0
NoiseComplaints             0
distance_to_facility        0
cultural_facility           0
education_facility          0
health_services             0
public_safety               0
recreational_facility       0
religious_institution       0
transportation_facility     0
DemographicYearMonth        0
population                  0
average_household_income    0
dtype: int64

In [391]:
unique_zipcodes = merged_df['zipcode'].nunique()
print(f"Number of unique NYC zip codes: {unique_zipcodes}")

Number of unique NYC zip codes: 166


In [392]:
merged_df

,zipcode,Borough,YearMonth,AveragePrice,Year,Month,ArrestCount,YearMonth,NoiseComplaints,distance_to_facility,cultural_facility,education_facility,health_services,public_safety,recreational_facility,religious_institution,transportation_facility,DemographicYearMonth,population,average_household_income
1,10001,Manhattan,2010-11-30,1591637,2010,11,101.0,2010-11-01,144.0,0.814942,0.0,89.0,0.0,0.0,0.0,0.0,0.0,2010-11-01,18158.000000,128706.559600
3,10001,Manhattan,2010-10-31,1600287,2010,10,125.0,2010-10-01,173.0,0.814942,0.0,89.0,0.0,0.0,0.0,0.0,0.0,2010-10-01,18158.000000,128706.559600
5,10001,Manhattan,2010-09-30,1587938,2010,9,122.0,2010-09-01,113.0,0.814942,0.0,89.0,0.0,0.0,0.0,0.0,0.0,2010-09-01,18158.000000,128706.559600
6,10001,Manhattan,2010-08-31,1566227,2010,8,126.0,2010-08-01,107.0,0.814942,0.0,89.0,0.0,0.0,0.0,0.0,0.0,2010-08-01,18158.000000,128706.559600
8,10001,Manhattan,2010-07-31,1543330,2010,7,144.0,2010-07-01,127.0,0.814942,0.0,89.0,0.0,0.0,0.0,0.0,0.0,2010-07-01,18158.000000,128706.559600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50384,11694,Queens,2012-03-31,548573,2012,3,41.0,2012-03-01,18.0,3.906032,0.0,0.0,0.0,0.0,0.0,0.0,89.0,2012-03-01,15130.000000,91399.022000
50387,11694,Queens,2023-05-31,847251,2023,5,7.0,2023-05-01,158.0,3.906032,0.0,0.0,0.0,0.0,0.0,0.0,89.0,2023-05-01,15520.288889,103495.462358
50388,11694,Queens,2015-09-30,558628,2015,9,36.0,2015-09-01,107.0,3.906032,0.0,0.0,0.0,0.0,0.0,0.0,89.0,2015-09-01,15740.000000,98029.659100
50391,11694,Queens,2014-10-31,525009,2014,10,32.0,2014-10-01,38.0,3.906032,0.0,0.0,0.0,0.0,0.0,0.0,89.0,2014-10-01,15270.000000,90293.571400


In [393]:
def filter_zero_values(merged_df, threshold=0.2):
    zero_count = (merged_df == 0).sum(axis=1)
    total_count = merged_df.shape[1]
    proportion_zero = zero_count / total_count
    return merged_df[proportion_zero <= threshold]

In [394]:
# Apply the filter to the property data
property_data_cols = [col for col in property_df.columns if 'YearMonth' not in col and 'zipcode' not in col]
filtered_property_df = filter_zero_values(property_df[property_data_cols])

# Only keep rows in merged_df that exist in the filtered property data
merged_df = merged_df[merged_df['zipcode'].isin(merged_df['zipcode'])]

In [395]:
merged_df

,zipcode,Borough,YearMonth,AveragePrice,Year,Month,ArrestCount,YearMonth,NoiseComplaints,distance_to_facility,cultural_facility,education_facility,health_services,public_safety,recreational_facility,religious_institution,transportation_facility,DemographicYearMonth,population,average_household_income
1,10001,Manhattan,2010-11-30,1591637,2010,11,101.0,2010-11-01,144.0,0.814942,0.0,89.0,0.0,0.0,0.0,0.0,0.0,2010-11-01,18158.000000,128706.559600
3,10001,Manhattan,2010-10-31,1600287,2010,10,125.0,2010-10-01,173.0,0.814942,0.0,89.0,0.0,0.0,0.0,0.0,0.0,2010-10-01,18158.000000,128706.559600
5,10001,Manhattan,2010-09-30,1587938,2010,9,122.0,2010-09-01,113.0,0.814942,0.0,89.0,0.0,0.0,0.0,0.0,0.0,2010-09-01,18158.000000,128706.559600
6,10001,Manhattan,2010-08-31,1566227,2010,8,126.0,2010-08-01,107.0,0.814942,0.0,89.0,0.0,0.0,0.0,0.0,0.0,2010-08-01,18158.000000,128706.559600
8,10001,Manhattan,2010-07-31,1543330,2010,7,144.0,2010-07-01,127.0,0.814942,0.0,89.0,0.0,0.0,0.0,0.0,0.0,2010-07-01,18158.000000,128706.559600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50384,11694,Queens,2012-03-31,548573,2012,3,41.0,2012-03-01,18.0,3.906032,0.0,0.0,0.0,0.0,0.0,0.0,89.0,2012-03-01,15130.000000,91399.022000
50387,11694,Queens,2023-05-31,847251,2023,5,7.0,2023-05-01,158.0,3.906032,0.0,0.0,0.0,0.0,0.0,0.0,89.0,2023-05-01,15520.288889,103495.462358
50388,11694,Queens,2015-09-30,558628,2015,9,36.0,2015-09-01,107.0,3.906032,0.0,0.0,0.0,0.0,0.0,0.0,89.0,2015-09-01,15740.000000,98029.659100
50391,11694,Queens,2014-10-31,525009,2014,10,32.0,2014-10-01,38.0,3.906032,0.0,0.0,0.0,0.0,0.0,0.0,89.0,2014-10-01,15270.000000,90293.571400


In [396]:
# Map zip codes to boroughs
zip_to_borough = {
    # Bronx
    '10453': 'Bronx', '10457': 'Bronx', '10460': 'Bronx',
    '10458': 'Bronx', '10467': 'Bronx', '10468': 'Bronx',
    '10451': 'Bronx', '10452': 'Bronx', '10456': 'Bronx',
    '10454': 'Bronx', '10455': 'Bronx', '10459': 'Bronx', '10474': 'Bronx',
    '10463': 'Bronx', '10471': 'Bronx',
    '10466': 'Bronx', '10469': 'Bronx', '10470': 'Bronx', '10475': 'Bronx',
    '10461': 'Bronx', '10462': 'Bronx', '10464': 'Bronx', '10465': 'Bronx', '10472': 'Bronx', '10473': 'Bronx',

    # Brooklyn
    '11212': 'Brooklyn', '11213': 'Brooklyn', '11216': 'Brooklyn', '11233': 'Brooklyn', '11238': 'Brooklyn',
    '11209': 'Brooklyn', '11214': 'Brooklyn', '11228': 'Brooklyn',
    '11204': 'Brooklyn', '11218': 'Brooklyn', '11219': 'Brooklyn', '11230': 'Brooklyn',
    '11234': 'Brooklyn', '11236': 'Brooklyn', '11239': 'Brooklyn',
    '11223': 'Brooklyn', '11224': 'Brooklyn', '11229': 'Brooklyn', '11235': 'Brooklyn',
    '11201': 'Brooklyn', '11205': 'Brooklyn', '11215': 'Brooklyn', '11217': 'Brooklyn', '11231': 'Brooklyn',
    '11203': 'Brooklyn', '11210': 'Brooklyn', '11225': 'Brooklyn', '11226': 'Brooklyn',
    '11207': 'Brooklyn', '11208': 'Brooklyn',
    '11211': 'Brooklyn', '11222': 'Brooklyn',
    '11220': 'Brooklyn', '11232': 'Brooklyn',
    '11206': 'Brooklyn', '11221': 'Brooklyn', '11237': 'Brooklyn',

    # Manhattan
    '10026': 'Manhattan', '10027': 'Manhattan', '10030': 'Manhattan', '10037': 'Manhattan', '10039': 'Manhattan',
    '10001': 'Manhattan', '10011': 'Manhattan', '10018': 'Manhattan', '10019': 'Manhattan', '10020': 'Manhattan', '10036': 'Manhattan',
    '10029': 'Manhattan', '10035': 'Manhattan',
    '10010': 'Manhattan', '10016': 'Manhattan', '10017': 'Manhattan', '10022': 'Manhattan',
    '10012': 'Manhattan', '10013': 'Manhattan', '10014': 'Manhattan',
    '10004': 'Manhattan', '10005': 'Manhattan', '10006': 'Manhattan', '10007': 'Manhattan', '10038': 'Manhattan', '10280': 'Manhattan',
    '10002': 'Manhattan', '10003': 'Manhattan', '10009': 'Manhattan',
    '10021': 'Manhattan', '10028': 'Manhattan', '10044': 'Manhattan', '10065': 'Manhattan', '10075': 'Manhattan', '10128': 'Manhattan',
    '10023': 'Manhattan', '10024': 'Manhattan', '10025': 'Manhattan',
    '10031': 'Manhattan', '10032': 'Manhattan', '10033': 'Manhattan', '10034': 'Manhattan', '10040': 'Manhattan',

    # Queens
    '11361': 'Queens', '11362': 'Queens', '11363': 'Queens', '11364': 'Queens',
    '11354': 'Queens', '11355': 'Queens', '11356': 'Queens', '11357': 'Queens', '11358': 'Queens', '11359': 'Queens', '11360': 'Queens',
    '11365': 'Queens', '11366': 'Queens', '11367': 'Queens',
    '11412': 'Queens', '11423': 'Queens', '11432': 'Queens', '11433': 'Queens', '11434': 'Queens', '11435': 'Queens', '11436': 'Queens',
    '11101': 'Queens', '11102': 'Queens', '11103': 'Queens', '11104': 'Queens', '11105': 'Queens', '11106': 'Queens',
    '11374': 'Queens', '11375': 'Queens', '11379': 'Queens', '11385': 'Queens',
    '11691': 'Queens', '11692': 'Queens', '11693': 'Queens', '11694': 'Queens', '11695': 'Queens', '11697': 'Queens',
    '11004': 'Queens', '11005': 'Queens', '11411': 'Queens', '11413': 'Queens', '11422': 'Queens', '11426': 'Queens', '11427': 'Queens', '11428': 'Queens', '11429': 'Queens',
    '11414': 'Queens', '11415': 'Queens', '11416': 'Queens', '11417': 'Queens', '11418': 'Queens', '11419': 'Queens', '11420': 'Queens', '11421': 'Queens',
    '11368': 'Queens', '11369': 'Queens', '11370': 'Queens', '11372': 'Queens', '11373': 'Queens', '11377': 'Queens', '11378': 'Queens',

    # Staten Island
    '10302': 'Staten Island', '10303': 'Staten Island', '10310': 'Staten Island',
    '10306': 'Staten Island', '10307': 'Staten Island', '10308': 'Staten Island', '10309': 'Staten Island', '10312': 'Staten Island',
    '10301': 'Staten Island', '10304': 'Staten Island', '10305': 'Staten Island',
    '10314': 'Staten Island'
}

# Map zip codes to boroughs
def get_borough(zipcode):
    str_zip = str(zipcode).zfill(5)  
    return zip_to_borough.get(str_zip, 'Unknown') 

merged_df = merged_df.drop(columns=['Borough'])

# Apply 
merged_df['Borough'] = merged_df['zipcode'].apply(get_borough)

In [397]:
merged_df

,zipcode,YearMonth,AveragePrice,Year,Month,ArrestCount,YearMonth,NoiseComplaints,distance_to_facility,cultural_facility,education_facility,health_services,public_safety,recreational_facility,religious_institution,transportation_facility,DemographicYearMonth,population,average_household_income,Borough
1,10001,2010-11-30,1591637,2010,11,101.0,2010-11-01,144.0,0.814942,0.0,89.0,0.0,0.0,0.0,0.0,0.0,2010-11-01,18158.000000,128706.559600,Manhattan
3,10001,2010-10-31,1600287,2010,10,125.0,2010-10-01,173.0,0.814942,0.0,89.0,0.0,0.0,0.0,0.0,0.0,2010-10-01,18158.000000,128706.559600,Manhattan
5,10001,2010-09-30,1587938,2010,9,122.0,2010-09-01,113.0,0.814942,0.0,89.0,0.0,0.0,0.0,0.0,0.0,2010-09-01,18158.000000,128706.559600,Manhattan
6,10001,2010-08-31,1566227,2010,8,126.0,2010-08-01,107.0,0.814942,0.0,89.0,0.0,0.0,0.0,0.0,0.0,2010-08-01,18158.000000,128706.559600,Manhattan
8,10001,2010-07-31,1543330,2010,7,144.0,2010-07-01,127.0,0.814942,0.0,89.0,0.0,0.0,0.0,0.0,0.0,2010-07-01,18158.000000,128706.559600,Manhattan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50384,11694,2012-03-31,548573,2012,3,41.0,2012-03-01,18.0,3.906032,0.0,0.0,0.0,0.0,0.0,0.0,89.0,2012-03-01,15130.000000,91399.022000,Queens
50387,11694,2023-05-31,847251,2023,5,7.0,2023-05-01,158.0,3.906032,0.0,0.0,0.0,0.0,0.0,0.0,89.0,2023-05-01,15520.288889,103495.462358,Queens
50388,11694,2015-09-30,558628,2015,9,36.0,2015-09-01,107.0,3.906032,0.0,0.0,0.0,0.0,0.0,0.0,89.0,2015-09-01,15740.000000,98029.659100,Queens
50391,11694,2014-10-31,525009,2014,10,32.0,2014-10-01,38.0,3.906032,0.0,0.0,0.0,0.0,0.0,0.0,89.0,2014-10-01,15270.000000,90293.571400,Queens


In [398]:
merged_df.to_csv('data/training_data.csv', index=False)

In [399]:
# Ensure unique column names
def make_column_names_unique(df):
    cols = pd.Series(df.columns)
    for dup in cols[cols.duplicated()].unique():
        cols[cols[cols == dup].index.values.tolist()] = [dup + '_' + str(i) if i != 0 else dup for i in range(sum(cols == dup))]
    df.columns = cols
    return df

# Apply the function to the merged_df
merged_df = make_column_names_unique(merged_df)

In [400]:
# Save to JSON
merged_df.to_json('data/training_data.json', orient='records', indent=4)